In [4]:
from dotenv import load_dotenv
import os
import uuid

load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END
from langgraph.prebuilt.tool_executor import ToolExecutor
from typing import TypedDict, Annotated
from operator import add

# LLM 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", 
    temperature=0.1,
)

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

# 그래프 정의
def chat_node(state):
    messages = state["messages"]
    question = state["question"]
    
    # 프롬프트 준비
    prompt_value = prompt.invoke({"history": messages, "question": question})
    
    # 응답 생성
    response = llm.invoke(prompt_value)
    
    # 상태 업데이트
    new_messages = messages + [HumanMessage(content=question), response]
    return {"messages": new_messages, "response": response.content}

# 그래프 생성
class State(TypedDict):
    messages: Annotated[list, add]
    question: str

workflow = StateGraph(State)

# 그래프에 노드 추가
workflow.add_node("chat", chat_node)

# 진입점 설정
workflow.set_entry_point("chat")

# 종료점 설정
workflow.add_edge("chat", END)

# 그래프 컴파일
app = workflow.compile()

# 지속성을 위한 MemorySaver 생성
memory_saver = MemorySaver()

# 그래프를 실행하는 함수
def run_conversation(question: str):
    session_id = str(uuid.uuid4())
    state = {"messages": [], "question": question}
    result = app.invoke(state, {"memory_saver": memory_saver, "session_id": session_id})
    return result["response"]

# 사용 예시
response = run_conversation("제 이름은 니코입니다")
print(response)


KeyError: 'response'